<a href="https://colab.research.google.com/github/henrycgbaker/Debt_Settlement_App/blob/main/uncertainty_celeba_TTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
from PIL import Image
from tqdm import tqdm
import timm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [3]:
# Download and extract dataset
!gdown 1Ny3af7iB1n3y0QYf9vpBN_x76Ti2MRtW # NB this is not the same as the one Carol has in latest uncertainty_celeba.ipynb
!unzip celeba_300.zip
!rm celeba_300.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Ny3af7iB1n3y0QYf9vpBN_x76Ti2MRtW
From (redirected): https://drive.google.com/uc?id=1Ny3af7iB1n3y0QYf9vpBN_x76Ti2MRtW&confirm=t&uuid=e1413add-ce7a-443f-ae1c-fa134025b593
To: /content/celeba_300.zip
100% 66.3M/66.3M [00:01<00:00, 42.1MB/s]
Archive:  celeba_300.zip
replace celeba_300/dataset_info.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Dataset

In [2]:
class CelebA300Dataset(Dataset):
    def __init__(self, root_dir='celeba_300'):
        # Load metadata
        info = torch.load(os.path.join(root_dir, 'dataset_info.pt'))
        self.metadata = info['metadata']
        self.n_classes = info['n_classes']
        self.root_dir = root_dir

        # Define transforms
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_data = self.metadata[idx]
        img_path = os.path.join(self.root_dir, 'images/img_align_celeba', img_data['img_name'])

        # Load and transform image
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)

        return image, img_data['identity']

# Create dataset and splits
def create_data_loaders(batch_size=32, train_split=0.8):
    dataset = CelebA300Dataset()

    # Calculate splits
    train_size = int(train_split * len(dataset))
    test_size = len(dataset) - train_size

    # Create splits
    train_dataset, test_dataset = random_split(
        dataset,
        [train_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # Create loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    print(f"Training samples: {len(train_dataset)}")
    print(f"Testing samples: {len(test_dataset)}")

    return train_loader, test_loader, dataset.n_classes

## Training

In [5]:
# Set up model, training and uncertainty

class FaceClassifier(nn.Module):
    def __init__(self, num_classes, dropout_rate=0):
        super().__init__()
        # Load model from timm
        self.model = timm.create_model(
            'mobilenetv3_small_100',
            pretrained=True,
            num_classes=num_classes,
            drop_rate=dropout_rate
        )

        # Add extra dropout for uncertainty
        self.model.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            self.model.classifier
        )

    def forward(self, x):
        return self.model(x)

    def enable_dropout(self):
        for m in self.model.modules():
            if isinstance(m, nn.Dropout):
                m.train()

# Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc='Training'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return running_loss / len(train_loader), accuracy

# Evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return running_loss / len(test_loader), accuracy

In [6]:
# Setup training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, num_classes = create_data_loaders(batch_size=32)

model = FaceClassifier(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2)

# Training loop
num_epochs = 10
best_accuracy = 0

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    # Adjust learning rate
    scheduler.step(test_acc)

    # Save best model
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        torch.save(model.state_dict(), 'best_face_model.pt')

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print('-' * 50)

<ipython-input-2-43d61689973a>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  info = torch.load(os.path.join(root_dir, 'dataset_info.pt'))


Training samples: 7230
Testing samples: 1808


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Evaluating: 100%|██████████| 57/57 [00:05<00:00, 10.97it/s]


...

Epoch 8/10:
Train Loss: 1.1066, Train Acc: 68.52%
Test Loss: 3.0978, Test Acc: 35.90%
--------------------------------------------------


Evaluating: 100%|██████████| 57/57 [00:04<00:00, 11.53it/s]


Epoch 9/10:
Train Loss: 0.8665, Train Acc: 74.81%
Test Loss: 2.8987, Test Acc: 41.04%
--------------------------------------------------


Evaluating: 100%|██████████| 57/57 [00:04<00:00, 11.89it/s]


Epoch 10/10:
Train Loss: 0.6641, Train Acc: 80.35%
Test Loss: 2.9435, Test Acc: 45.13%
--------------------------------------------------


## Test-Time Augmentation (TTA)

Here’s how to implement Test-Time Augmentation (TTA) for uncertainty estimation using a pre-trained model. TTA involves applying augmentations to the input image during inference and analyzing the variability in predictions.

Here, uncertainty is the standard deviation of predictions across augmented images (TTA).

Steps:

* Generate predictions for each augmentation.
* Aggregate predictions into a probability distribution for each class.
* Calculate the standard deviation across the predictions for the augmented images.

In [11]:
import ttach as tta
from torchvision import transforms

# Define image transformations (match your dataset's preprocessing)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load a single image
image_path = "celeba_300/images/img_align_celeba/157647.jpg"  # Replace with your image path
image = Image.open(image_path)
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Move image to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)

# Wrap the model with TTA
tta_transforms = tta.Compose([
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 90, 180, 270]),
])

# Store predictions for all augmentations
model.eval()
all_predictions = []

with torch.no_grad():
    for transform in tta_transforms:
        # Apply each augmentation
        augmented_tensor = transform.augment_image(input_tensor)
        outputs = model(augmented_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        all_predictions.append(probabilities.cpu().numpy())

# Convert list to a numpy array for easy calculations
all_predictions = np.array(all_predictions)  # Shape: [num_augmentations, batch_size, num_classes]

# Calculate mean prediction across augmentations
mean_prediction = np.mean(all_predictions, axis=0)  # Shape: [batch_size, num_classes]

# Calculate uncertainty as standard deviation across augmentations
uncertainty = np.std(all_predictions, axis=0)  # Shape: [batch_size, num_classes]

# Display results
predicted_class = np.argmax(mean_prediction, axis=1).item()
uncertainty_score = uncertainty[0, predicted_class]

print(f"Predicted class: {predicted_class}")
print(f"Uncertainty score (std-dev for class {predicted_class}): {uncertainty_score:.4f}")


Predicted class: 31
Uncertainty score (std-dev for class 31): 0.3481
